# CoMET API Notebook Standalone

This notebook has all API calls embedded in the notebook and does not require any supporting imports. Each method code row can be ran independently.

## Login 
To use the rest of this code, you must login.
- You must have a CoMET account created for you
- Only users with a noaa.gov email are given CoMET accounts
- You must be assigned a Record Group
- You must have the correct permissions for that Record Group

In [8]:
# Login to API endpoint
# This will setup your user session which is required for all endpoints
import requests
import getpass
import json
from urllib.parse import urlparse
from os.path import exists
import pandas as pd
from pandas.io.json import json_normalize

def login ():
    session = requests.Session()

    LOGIN_URL = "https://data.noaa.gov/cedit/login/wsLogin"
    username = input("Enter Username: ")
    password = getpass.getpass(prompt='Enter password: ')

    PARAMS = {'username': username, 'password': password}
    resp = session.post(LOGIN_URL, params = PARAMS)

    print(resp.text)

    # Saves Session to disk for use in other python examples
    if resp.text == "success": 
        print(session.cookies)
        print("You successfully logged in");

        # with open('comet_cookie.txt', 'w') as f:
        #     json.dump(requests.utils.dict_from_cookiejar(session.cookies), f)
    else:
        print("login failed")
    password = ""
    return session

session = login()

success
<RequestsCookieJar[<Cookie JSESSIONID=B3643BBB7BA588B7E9D96F5BDC1932E4 for data.noaa.gov/cedit>]>
You successfully logged in


### <a id="search"></a>Search a Record Group
* * *

This function will search for records within a Record Group and return each record's UUID, file identifier, and record name.

#### Parameters


Name | Type | Description  | Notes
------------- | ------------- | ------------- | -------------
**recordGroup** | **str**| The record group, which may represent a Web Accessible Folder (WAF) that contains 1 or more ISO metadata records stored in CoMET. | **Required**
**editState** | **str**| Status of record. | optional 
**format** | **str**| Response format. This parameter does not change the format of metadata documents/data. Use the transform parameter instead | optional [default to json]
**max** | **int**| Maximum number of records to return | optional [default to 5000]
**offset** | **int**| Number of records to exclude from the responses first N items. Use with max to request a set of items. Offset is applied before the max parameter regardless of its position in the request.  | optional [default to 1]
**since** | **date**| Return metadata records updated since this date. Requires yyyy-MM-dd format | optional 

Uncomment PARAMS lines to add variables...

In [10]:
# Setup Search Params
# Edit what parameters you want to include here. 
# Either type in a value in between ' ', or be prompted for a value when you run the cell
# PARAMS not included in query have the # mark in front of it
# Hit play before running next line of code

def set_search_PARAMS ():
    PARAMS = {
        'recordGroup': '', # required - example: ea_demo/
        # 'editState': '',
        # 'since': '', 
        # 'max': '',
        # 'offset': ,
        'format': "json"
    }

    for k, v in PARAMS.items():
        if v == '':
            value = str(input('Enter a value for {} '.format(k)));
            PARAMS[k] = value

    print(">>> Search query parameters set. You can continue....")
    return PARAMS

search_PARAMS = set_search_PARAMS()

>>> Search query parameters set. You can continue....


In [27]:
# Search for records by Record Group
# Make sure to setup search parameters above before running
def search (search_PARAMS):
    if not session:
        print ("Please Login")
    else:
        print(">>> Searching reecords with these PARAMS: ", search_PARAMS)
        SEARCH_URL = "https://data.noaa.gov/cedit/metadata/search"

        resp = session.get(SEARCH_URL, params = search_PARAMS)
        domain = urlparse(resp.url).netloc

        print(">> Searching Record Group: " + search_PARAMS["recordGroup"])
        if "auth" in domain:
            print("Your session expired...please login")
        else:
            print(resp.status_code) 
            return resp.text
    

try:
    search_results = search(search_PARAMS)

    # Display search results in table format
    loadJson = json.loads(search_results)
    records = pd.json_normalize(loadJson["metadata"]) 
    df = pd.DataFrame.from_dict(records)

    # Show UUID First
    df.set_index(df.columns[-1], inplace=True)
    df.reset_index(inplace=True)

    styles = [
        dict(selector='th', props=[('text-align', 'left')]),
        dict(selector='tr', props=[('text-align', 'left')]),
        dict(selector='td:nth-child(2)', props=[('white-space', 'nowrap')]),
    ]

    html = (df.style.set_table_styles(styles))
    display(html)
except:
    print('please setup search parameters first')

>>> Searching reecords with these PARAMS:  {'recordGroup': 'ea_demo/', 'format': 'json'}
>> Searching Record Group: ea_demo/
200


,uuid,fileIdentifier,name
0,5e9b3577-9521-4dc5-ae2b-640c390d55c0,api:testing:file:005,Test Import
1,872a1c21-fecf-4cdd-b2b6-633f5f951e1d,api:testing:file:003,Test import 2
2,42e88d79-22c8-4767-9820-21ee369e4866,gov.noaa.ncei:test_import_20,TestImport_12
3,e0b7acbb-fee1-4570-8fcb-435f1513fde6,gov.noaa.ncei:test_import_13,TestImport_13
4,8273f765-3639-4bfd-8a6a-f3a68ea3b187,gov.noaa.ncei:test_import_10_txt_Corruptformat_Param,TestImport_12
5,5eb81e29-fcb9-4a78-a0d1-282493e3288d,gov.noaa.ncei:test_import_10_txt_format_Param,TestImport_10_format_text
6,37817024-fe15-4748-acc3-ac4a89d5acfb,gov.noaa.ncei:test_import_10_xml_format_Param,TestImport_10_format_xml
7,0add6a13-3a20-4fa2-9c5d-94397eb8de9c,gov.noaa.ncei:test_import_10_json_format_Param,TestImport_10_format_json
8,f80c22e6-6205-4f78-96df-0a98146d7ab9,gov.noaa.ncei:test_import_6,TestImport_1
9,1d2b290b-abe9-479b-8179-a5dd819bdf90,gov.noaa.ncei:test_import_3,TestImport_1


### <a id="validate"></a>Validate a Record
* * *

This function will take a record located in the uploads folder and validate it according to the ISO 19115-2 standard

#### Parameters

Name | Type | Description  | Notes
------------- | ------------- | ------------- | -------------
 **uuid** | **str**| Universally Unique Identifier. Use this query parameter if you create your own UUIDs. Otherwise, the system will generate one for you.  | **Required**
 **transform** | **str**| Type of transformation to apply to the metadata record. Please review the section title Transformations for more detail. | **Required** [defaults to convert-comet-to-iso19115-2]
 **format** | **str**| Response format. This parameter does not change the format of metadata documents/data. Use the transform parameter instead | optional [defaults to json]] 

In [20]:
# Setup Validate Params
# Edit what parameters you want to include here. 
# Either type in a value in between ' ', or be prompted for a value when you run the cell
# PARAMS not included in query have the # mark in front of it
# Hit play before running next line of code

def set_validate_PARAMS ():
    PARAMS = {
        'uuid': '', # required - run search to get a UUID 
        'transform': 'convert-comet-to-iso19115-2', # required - recommend leaving as is, do not remove
        'format': "json"
    }

    for k, v in PARAMS.items():
        if v == '':
            value = str(input('Enter a value for {} '.format(k)));
            PARAMS[k] = value

    print(">>> Validate query parameters set. You can continue....")
    return PARAMS

validate_PARAMS = set_validate_PARAMS()

>>> Validate query parameters set. You can continue....


In [21]:
# Validate one record

def validate(validate_PARAMS):
    
    print(">>> Validate reecord with these PARAMS: ", validate_PARAMS)
    VALIDATE_URL = "https://data.noaa.gov/cedit/metadata/validate/"

    if not session:
        print("Please login")
    else:     
        resp = session.get(VALIDATE_URL+ validate_PARAMS["uuid"] , params=validate_PARAMS)
        domain = urlparse(resp.url).netloc

        print(">> Validating UUID: " + validate_PARAMS["uuid"])
        if "auth" in domain:
            print("Your session expired...please login")
        else: 
            return resp.text
    
try:
    validate_results = validate(validate_PARAMS)
    print(validate_results)
except:    
    print('please setup validate parameters first')

>>> Validate reecord with these PARAMS:  {'uuid': '872a1c21-fecf-4cdd-b2b6-633f5f951e1d', 'transform': 'convert-comet-to-iso19115-2', 'format': 'json'}
>> Validating UUID: 872a1c21-fecf-4cdd-b2b6-633f5f951e1d
{"message":"success","status":200}


### <a id="create"></a>Import/Create
* * * 
Create new new metadata record. Expects and ISO 19115-2 xml document
- The fileIdentifer and UUID in the xml file must be unique. If a record in the database already has this record -- regardless of whether that record is in your record group -- the import/create will fail

#### Parameters

Name | Type | Description  | Notes
------------- | ------------- | ------------- | -------------
 **body** | **document** | Accepts XML document in Comet Format XML or ISO 19115-2 XML. Default Comet Format XML. To change, use the transform parameter | ***Required***
 **record_group** | **str** | The record group, which may represent a Web Accessible Folder (WAF) that contains 1 or more ISO metadata records stored in CoMET. | ***Required***
 **description** | **str** | Title of the record in the CoMET database | ***Required***
 **transform** | **str**| Type of transformation to apply to the metadata record. Please review the section title Transformations for more detail. | ***Required*** [defaults to convert-iso19115-2-comet]
 **uuid** | **str** | Universally Unique Identifier. For import, Use this query parameter if you create your own UUIDs. Otherwise, the system will generate one for you.  | optional
 **edit_state** | **str** | Status of record. | optional
 **format** | **str**| Response format. This parameter does not change the format of metadata documents/data. Use the transform parameter instead | optional [defaults to json]

In [28]:
# Setup Import/Create Params
# Edit what parameters you want to include here. 
# Either type in a value in between ' ', or be prompted for a value when you run the cell
# PARAMS not included in query have the # mark in front of it
# Hit play before running next line of code

def set_create_PARAMS ():
    PARAMS = {
        'recordGroup': '', # Required - example: ea_demo/
        'description': '', # Required - Enter title of record for the CoMET database
        'transform': 'convert-iso19115-2-to-comet', # Required - Recommended to leave this value as is
        # 'uuid': '', # If you create your own UUIDs, uncomment; otherwise one will be created for you
        # 'editState': '', # System assigns DRAFT to all imports. Override with this parameter
        'format': 'json'
    }

    for k, v in PARAMS.items():
        if v == '':
            value = str(input('Enter a value for {} '.format(k)));
            PARAMS[k] = value
    print(">>> Import/Create query parameters set. You can continue....")
    return PARAMS

create_PARAMS = set_create_PARAMS()

>>> Import/Create query parameters set. You can continue....


In [30]:
# Import a single record

def create (create_PARAMS):
    
    print(">>> Import/Create reecord with these PARAMS: ", create_PARAMS)
    IMPORT_URL = "https://data.noaa.gov/cedit/metadata/import"

    if not session:
        print("Please Login")
    else:
        filename = input("Enter the filename (including path if located in a different directory): ")

        # if the file cannot be found, don't do anything
        file_exists = exists(filename)
        if not file_exists:
            print("Could not locate the file")
        else: 
            headers = {'Content-Type': 'application/xml'}

            with open(filename, 'rb') as f:
                print(">> Attempting to import " + filename + " into record group" + create_PARAMS['recordGroup'])
                resp = session.post(IMPORT_URL, headers = headers, params = create_PARAMS, data = f)
                
            domain = urlparse(resp.url).netloc
            if "auth" in domain:
                print("Your session expired...please login")
            else:
                return(resp.text)

try:
    create_results = create(create_PARAMS)
    print(create_results)
except:    
    print('please setup validate parameters first')                    

>>> Import/Create reecord with these PARAMS:  {'recordGroup': '', 'description': '', 'transform': 'convert-iso19115-2-to-comet', 'format': 'json'}
>> Attempting to import test-export.xml into record group
{"message":"Validation of request failed: Property [recordGroup] of class [class cedit.CreateMetadataRequest] cannot be blank; Property [description] of class [class cedit.CreateMetadataRequest] cannot be blank;","status":400,"uuid":null}


### <a id="delete"></a>Delete a Record
* * *
#### Parameters

Name | Type | Description  | Notes
------------- | ------------- | ------------- | -------------
 **uuid** | **str**| Universally Unique Identifier assigned to the record | ***Required***
 **format** | **str**| Response format. This parameter does not change the format of metadata documents/data. Use the transform parameter instead | optional

In [ ]:
# Setup Delete Params
# Edit what parameters you want to include here. 
# Either type in a value in between ' ', or be prompted for a value when you run the cell
# PARAMS not included in query have the # mark in front of it
# Hit play before running next line of code

def set_delete_PARAMS(): 
    PARAMS = {
        'uuid': '', # Required - UUID of the Record
        'format': "json"
    }

    for k, v in PARAMS.items():
            if v == '':
                value = str(input('Enter a value for {} '.format(k)));
                PARAMS[k] = value
    
    print(">>> Delete query parameters set. You can continue....")
    return PARAMS

delete_PARAMS = set_delete_PARAMS()

In [ ]:
# Delete a single record

def delete (delete_PARAMS):
    
    print(">>> Deleting reecord with these PARAMS: ", delete_PARAMS)
    DELETE_URL = "https://data.noaa.gov/cedit/metadata/"

    if not session:
        print("Please Login")
    else:
        print(">> Attempting delete of record: " + delete_PARAMS['uuid'])
        resp = session.delete(DELETE_URL+delete_PARAMS['uuid'], params=delete_PARAMS)
        domain = urlparse(resp.url).netloc

        if "auth" in domain:
            print("Your session expired...please login")
        else:
            return resp.text

try:
    delete_results = delete(delete_PARAMS)
    print(delete_results)
except:    
    print('please setup validate parameters first')

### <a id="update"></a>Update a Record
* * *
Update your metadata record. Expects and ISO 19115-2 xml documennt and UUID of the record
#### Parameters

Name | Type | Description  | Notes
------------- | ------------- | ------------- | -------------
 **body** | **document**  | Accepts XML document in Comet Format XML or ISO 19115-2 XML. Default Comet Format XML. To change, use the transform parameter | **Required**
 **transform** | **str**  | Type of transformation to apply to the metadata record. Please review the section title Transformations for more detail. | **Required** [defaults to convert-iso19115-2-to-comet]
 **uuid** | **str**| Universally Unique Identifier.   | **Required** 
 **description** | **str**| Title of the record in the CoMET database. Including this parameter will edit the title of your record | optional 
 **edit_state** | **str**| Status of record. | optional 
 **format** | **str**| Response format. This parameter does not change the format of metadata documents/data. Use the transform parameter instead | optional [defaults to json] 

In [ ]:
# Setup Update Params
# Edit what parameters you want to include here. 
# Either type in a value in between ' ', or be prompted for a value when you run the cell
# PARAMS not included in query have the # mark in front of it
# Hit play before running next line of code

def set_update_PARAMS(): 
    PARAMS = {
        'uuid': '', # Required
        # 'description': '', # Update title of record for the CoMET database
        'transform': 'convert-iso19115-2-to-comet', # Required - Recommended to leave this value as is
        # 'editState': '', # System assigns DRAFT to all imports. Over-ride with this parameter
        'format': 'json'
    }

    for k, v in PARAMS.items():
        if v == '':
            value = str(input('Enter a value for {} '.format(k)));
            PARAMS[k] = value

    print(">>> Update query parameters set. You can continue....")
    return PARAMS

update_PARAMS = set_update_PARAMS()

In [ ]:
import requests
import json
from urllib.parse import urlparse
from os.path import exists

def update(update_PARAMS):
    
    print(">>> Updating with these PARAMS: ", update_PARAMS)
    UPDATE_URL = "https://data.noaa.gov/cedit/metadata/"

    # if cookie can't be found, exit
    file_exists = exists('comet_cookie.txt')
    if not file_exists:
        print("Can't find cookie, please login")
    else:
        # Create a session and update it with your jsessionid saved in comet_cookie.txt
        session = requests.Session()
        with open('comet_cookie.txt', 'r') as f:
            cookies = requests.utils.cookiejar_from_dict(json.load(f))
            session.cookies.update(cookies)

        filename = input("Enter the updated filename (including path if located in a different directory): ")

        # if the file cannot be found, exit
        file_exists = exists(filename)
        if not file_exists:
            print("Could not locate the file")
        else:
            headers = {'Content-Type': 'application/xml'}

            with open(filename, 'rb') as f:
                resp = session.put(UPDATE_URL+update_PARAMS['uuid'], headers=headers, params=update_PARAMS, data = f)
                
            domain = urlparse(resp.url).netloc
            if "auth" in domain:
                print("Your session expired...please login")
            else:
                return resp.text

update_results = update(update_PARAMS)
print(update_results)

### <a id="export"></a>Export a Record
* * *

- Default export format is ISO 19115-2 

#### Parameters
Name | Type | Description  | Notes
------------- | ------------- | ------------- | -------------
 **uuid** | **str**| Universally Unique Identifier assigned to the record  | **Required**
 **transform** | **str**| Type of transformation to apply to the metadata record. Please review the section title Transformations for more detail. | optional [defaults to convert-comet-to-iso19115-2]
 **format** | **str**| Response format. This parameter does not change the format of metadata documents/data. Use the transform parameter instead | optional [defaults to json]
 

In [ ]:
# Setup Export Params
# Edit what parameters you want to include here. 
# Either type in a value in between ' ', or be prompted for a value when you run the cell
# PARAMS not included in query have the # mark in front of it
# Hit play before running next line of code

def set_export_PARAMS ():
    PARAMS = {
        'uuid': '', # Required 
        'transform': 'convert-comet-to-iso19115-2', # Required - Recommended to leave this value as is
        'format': "json"
    }

    for k, v in PARAMS.items():
        if v == '': # Required
            value = str(input('Enter a value for {} '.format(k)));
            PARAMS[k] = value
    
    print("Export query parameters set. You can continue....")
    return PARAMS

export_PARAMS = set_export_PARAMS ()

In [ ]:
# Export a Record and Save to dsik

def export(export_PARAMS):
    print(">>> Exporting with these PARAMS: ", export_PARAMS)

    EXPORT_URL = "https://data.noaa.gov/cedit/metadata/"

    if not session:
        print("Please Login")
    else:
        filename = input("Save file with this name: ")
        if filename != "":    
            resp = session.get(EXPORT_URL+export_PARAMS['uuid'], params=export_PARAMS)
            domain = urlparse(resp.url).netloc
        
            if "auth" in domain:
                print("Your session expired...please login")
            else:
                print(resp.status_code)
                if resp.status_code == 200:
                    with open(filename, 'wb') as f:
                        f.write(resp.content)
                    
                    return (">>> " + export_PARAMS['uuid'] + " was exported as " + filename)
                else:
                    print(resp.text)
        else:
            print("Could not export with blank filename")

try:
    export_results = export(export_PARAMS)
    print(export_results)
except:    
    print('please setup validate parameters first')

### Preview WAF Index Page

Clicking the play button below, will allow you to ***preview*** your WAF Index page. None of the links display will be clickable. You can, however, click the **Open WAF Index Page in Browser** link to open it in a browswer.  

In [ ]:
# Preview WAF Index Page
from IPython.display import IFrame, HTML

recordGroup = str(input('Enter Record Group Name'));

if recordGroup != "":
    display(HTML("""<h2><a href="https://data.noaa.gov/waf/"""+recordGroup+"""/iso/">Open WAF Index Page in Browser</a></h2>"""))    
    display(IFrame('https://data.noaa.gov/waf/'+recordGroup+'/iso/', width="100%", height="500px"))
else:
    print("You did not supply a record group name")
